In [23]:
import pandas as pd
r = pd.read_csv('result.csv', encoding='utf8')
r.head()

,Answer,Object,ObjectAnchor,Predicate,PredicateId,Subject,SubjectAnchor,Text,WikipediaLink,WikipediaTitle,_id
0,weak,Володимир Мономах,Володимиром Мономахом,батько,P22,Мстислав Великий,Мстиславом Великим,Ярополк жив у згоді з батьком Володимиром Моно...,https://uk.wikipedia.org/wiki/Ярополк Володими...,Ярополк Володимирович,0009f2f32483bbd4b68d
1,has,Флавій Орест,Флавій Орест,батько,P22,Ромул Август,Ромула Августа,Після смерті Ріцімера Східна Римська імперія с...,https://uk.wikipedia.org/wiki/V століття,V століття,001002218cb90f7de4ba
2,donthas,Уайна Капак,Уайни Капака,батько,P22,Уаскар,Уаскар,Був сином Сапа Інки Уайни Капака та молодшої д...,https://uk.wikipedia.org/wiki/Паулью Інка,Паулью Інка,001548301c08dc4d717f
3,has,Ярослав Мудрий,Ярослава I Мудрого,батько,P22,Володимир Ярославич (князь новгородський),Володимир Ярославич,"Володимир Ярославич — князь Новгородський, син...",https://uk.wikipedia.org/wiki/Володимир,Володимир,001809d2230282f77a33
4,donthas,Мехмед II Фатіх,Мехмеді ІІ,батько,P22,Баязид II,Баязиді ІІ,Відповідно до напису біля входу до приміщення ...,https://uk.wikipedia.org/wiki/Аладжа Імарет (С...,Аладжа Імарет (Салоніки),001915ebadd92442641d


In [24]:
r = r[r['Answer'] != 'not']
len(r)

5053

In [25]:
r['Answer'].value_counts()

has        2091
donthas    2032
weak        930
Name: Answer, dtype: int64

In [26]:
row = r.iloc[5042]
text = row['Text']
print text

У 102 до н. е. Гай Юлій одружується на Аврелії Котті. У шлюбі в них було троє дітей: дві дочки — Юлія Цезаріс Старша Юлія Цезаріс Молодша, народилася в 101 до н. е. — і син, Гай Юлій Цезар, який народився в 100 до н. е.


In [27]:
from tokenize_uk import *

tokens = tokenize_text(row['Text'])
tokens = [t for tt in tokens for sentence in tt for t in sentence]
print '|'.join(tokens)

У|102|до|н|.|е|.|Гай|Юлій|одружується|на|Аврелії|Котті|.|У|шлюбі|в|них|було|троє|дітей|:|дві|дочки|—|Юлія|Цезаріс|Старша|Юлія|Цезаріс|Молодша|,|народилася|в|101|до|н|.|е|.|—|і|син|,|Гай|Юлій|Цезар|,|який|народився|в|100|до|н|.|е|.


In [28]:
import re
objectAnchor = row['ObjectAnchor']
def detect_anchor(tokens, anchor):
    found = False
    index = 0
    for i in range(len(tokens)):
        token = tokens[i]
        pattern = re.escape(anchor[index]) + u"[а|ом|у]?"
        match = re.match(pattern, token, flags=re.U|re.L)
        
        if(found):
            if(match is not None):
                index += 1
            else:
                index = 0
                found = False
                pattern = re.escape(anchor[index]) + u"[а|ом]?"
                match = re.match(pattern, token, flags=re.U|re.L)
        
        if(not found and match is not None):
            start = i
            index += 1
            found = True
            
        if(index == len(anchor)):
            end = i + 1
            yield range(start, end)
            index = 0
            found = False
            


def extract_from_tokens_by_range(tokens, r):
    return ' '.join(tokens[r[0]:r[-1]+1])
    
detection = list(detect_anchor(tokens, tokenize_words(objectAnchor)))
print extract_from_tokens_by_range(tokens, detection[0])

Аврелії Котті


In [29]:
subjectAnchor = row['SubjectAnchor']
detection = list(detect_anchor(tokens, tokenize_words(subjectAnchor)))
detection

[[28, 29, 30]]

In [30]:
print extract_from_tokens_by_range(tokens, detection[0])

Юлія Цезаріс Молодша


In [31]:
def show_row_anchors_and_text(row, verbose=False):
    text = row['Text']
    if(verbose):
        print text
    tokens = tokenize_text(text)
    tokens = [t for tt in tokens for sentence in tt for t in sentence]
    subject_detection = list(detect_anchor(tokens, tokenize_words(row['SubjectAnchor'])))
    if(verbose):
        for d in subject_detection:
            print extract_from_tokens_by_range(tokens, d)
    object_detection = list(detect_anchor(tokens, tokenize_words(row['ObjectAnchor'])))
    if(verbose):
        for d in object_detection:
            print extract_from_tokens_by_range(tokens, d)
    return (subject_detection, object_detection)
        
show_row_anchors_and_text(r.iloc[0], True)

Ярополк жив у згоді з батьком Володимиром Мономахом і старшим братом Мстиславом Великим.
Мстиславом Великим
Володимиром Мономахом


([[11, 12]], [[6, 7]])

In [32]:
show_row_anchors_and_text(r.iloc[1], True)

Після смерті Ріцімера Східна Римська імперія спробувала прислати на захід свого імператора Юлія Непота, однак грецький імператор не отримав підтримки від римлян, і владу узурпував Флавій Орест, оголосивши імператором свого сина Ромула Августа.
Ромула Августа
Флавій Орест


([[34, 35]], [[27, 28]])

In [33]:
show_row_anchors_and_text(r.iloc[2], True)

Був сином Сапа Інки Уайни Капака та молодшої дружини Аньяс Кольке. Оскільки був доволі молодим, то не брав участі у боротьби міє Атауальпою і Уаскаром, а згодом проти іспанців.
Уаскаром
Уайни Капака


([[26]], [[4, 5]])

In [34]:
positions = r.apply(show_row_anchors_and_text, axis=1)

In [35]:
err = []
for i in range(len(positions)):
    try:
        p = positions.iloc[i]
    except:
        err.append(i)
    if(len(p[0]) == 0):
        err.append(i)
    if(len(p[1]) == 0):
        err.append(i)
err

[]